## Basic Model

### Initialization

In [1]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup as BS
import os
import pickle

path = r"C:\Users\yongz\Personal Projects\Side Proj\Side_Proj_Data"
os.chdir(path)

### Data Collection
Scrape site

In [26]:
# Steps
def generate_steps(raw):
    divs = raw.find_all('div')
    steps = []
    for i in range(len(divs)):
        if i < len(divs)-2:
            if (
                divs[i].has_attr('class')
                and divs[i+1].has_attr('class')
                and divs[i+2].has_attr('class')
                and divs[i].find('h3') is not None
            ):
                if (
                    divs[i]['class'][0] == "cmp-title"
                    and divs[i+1]['class'][0] == "text"
                    and 'Step' in divs[i].find('h3').text
                ):
                    steps.append([divs[i],divs[i+1]])
                elif (
                    len(divs[i].find_all('h4',class_='cmp-title__text')) > 0
                    and 'cmp-text' in divs[i+2]['class'][0]
                ):
                    steps.append([divs[i].find('h4'),divs[i+2]])
                elif (
                    'cmp-text' in divs[i]['class'][0]
                ):
                    title = divs[i].find('p')
                    content = [x for x in divs[i].find_all(['p','ol','h3','h4','ul']) if x!=title]
                    steps.append([title,content])
            elif (
                divs[i].has_attr('class')
                and divs[i+1].has_attr('class')
                and divs[i].find('h3') is not None
            ): 
                if (
                    divs[i]['class'][0] == "cmp-title"
                    and divs[i+1]['class'][0] == "text"
                    and 'Step' in divs[i].find('h3').text
                ):
                    steps.append([divs[i],divs[i+1]])
                elif (
                    'cmp-text' in divs[i]['class'][0]
                    and divs[i].find('p') is not None
                ):
                    title = divs[i].find('p')
                    content = [x for x in divs[i].find_all(['p','ol','h3','h4','ul']) if x!=title]
                    steps.append([title,content])
            elif (divs[i].has_attr('class')):
                 if (
                     'cmp-text' in divs[i]['class'][0]
                     and divs[i].find('p') is not None
                 ):
                    title = divs[i].find('p')
                    content = [x for x in divs[i].find_all(['p','ol','h3','h4','ul']) if x!=title]
                    steps.append([title,content])
        
        elif i < len(divs)-1:
            if (
                divs[i].has_attr('class')
                and divs[i+1].has_attr('class')
                and divs[i].find('h3') is not None
            ): 
                if (
                    divs[i]['class'][0] == "cmp-title"
                    and divs[i+1]['class'][0] == "text"
                    and 'Step' in divs[i].find('h3').text
                ):
                    steps.append([divs[i],divs[i+1]])
                elif (
                    'cmp-text' in divs[i]['class'][0]
                    and divs[i].find('p') is not None
                ):
                    title = divs[i].find('p')
                    content = [x for x in divs[i].find_all(['p','ol','h3','h4','ul']) if x!=title]
                    steps.append([title,content])
            elif (divs[i].has_attr('class')):
                 if (
                     'cmp-text' in divs[i]['class'][0]
                     and divs[i].find('p') is not None
                 ):
                    title = divs[i].find('p')
                    content = [x for x in divs[i].find_all(['p','ol','h3','h4','ul']) if x!=title]
                    steps.append([title,content])
        elif(divs[i].has_attr('class')):
            if (
                'cmp-text' in divs[i]['class'][0]
                and divs[i].find('p') is not None
            ):
                title = divs[i].find('p')
                content = [x for x in divs[i].find_all(['p','ol','h3','h4','ul']) if x!=title]
                steps.append([title,content])
    
    return steps

In [27]:
# FAQ
def generate_faq(raw):
    spans = raw.find_all('span')
    faq = []
    for i in range(len(spans)):
        try:
            if (spans[i]['class'][0] == 'leo-toggle--title'
                and spans[i+2]['class'][0] == 'footnotes-icon-help'
            ):
                faq.append([spans[i],spans[i+2]])
        except:
            continue
    return faq

In [28]:
def process_tags(tag_list):
    symbols_nospace = ['<i>','</i>',u'\t']
    symbols_space = ['<b>','</b>',u'\n',u'\xa0']
    processed = []
    for t in tag_list:
        if t[0] is None or t[1] is None: continue
        title = t[0].text
        if isinstance(t[1],list) == False:
            content = t[1].find_all(['p','h3','h4','li'])
        else:
            content = []
            for c in t[1]:
                if len(c.find_all()) > 0:
                    content.extend(c.find_all(['p','li']))
                else: 
                    content.append(c)
        combined = title
        for line in content:
            line = line.text
            for s in symbols_nospace:
                line = line.replace(s,'')
            for s in symbols_space:
                line = line.replace(s,' ') #.rstrip()
            combined += ' '
            combined += line
        processed.append(combined)
    return processed

In [61]:
def get_data(links_dict):
    df = pd.DataFrame(columns=['page','text'])
    for page in links_dict:
        req = requests.get(links_dict[page])
        raw = BS(req.content,'html.parser')
        steps_page = process_tags(generate_steps(raw))
        steps_page = [page.split('/')[-1] + ' steps: ' + x for x in steps_page]
        faq_page = process_tags(generate_faq(raw))
        faq_page = [page.split('/')[-1] + ' FAQ: ' + x for x in faq_page]
        temp_df = pd.DataFrame(steps_page,columns=['text'])
        temp_df = pd.concat([temp_df,pd.DataFrame(faq_page,columns=['text'])],ignore_index=True)
        temp_df['page'] = page
        temp_df = temp_df[['page','text']]
        df = pd.concat([df,temp_df],ignore_index=True)
    return df

#### Complete Run:

Claims:

In [2]:
# Generate list of webpages for GE/Self-service guides/Claims
claims_r = requests.get('https://www.greateasternlife.com/sg/en/customer-services/guides.html?topic=claims')
claims_raw = BS(claims_r.content,'html.parser').find_all('div',class_='claims')

links_0 = claims_raw[1].find_all('a',class_='leo-button leo-font-normal leo-border-b-0 leo-button--link leo-link-arrow')
claims_links = {}
for link in links_0:
    try:
        if ('form' not in link['title']
            and ' claim' in link['title']
           ):
            link_title = 'Self-service guides/Claims/'+link['data-subcategory']
            claims_links[link_title] = 'https://www.greateasternlife.com'+link['href']
    except:
        continue

In [30]:
claims_df = get_data(claims_links)
claims_df.head(10)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:18<00:00,  1.42s/it]


,page,text
0,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim steps: If you...
1,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim steps: \nStep...
2,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim steps: \nStep...
3,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: Need mor...
4,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: What is ...
5,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: What are...
6,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: Can I se...
7,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: Should I...
8,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: How long...
9,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: How will...


Investment-linked:

In [22]:
inv_r = requests.get('https://www.greateasternlife.com/sg/en/customer-services/guides.html?topic=insurance-guides')
inv_raw = BS(inv_r.content,'html.parser').find('div',class_='service-guide-inside-list d-none investment-linked')
links_0 = inv_raw.find_all('a',class_='leo-button leo-font-normal leo-border-b-0 leo-button--link leo-link-arrow')
inv_links = {}
for link in links_0:
    try:
        if ('View' not in link['title']
           ):
            link_title = 'Self-service guides/Investment-linked/'+link['title']
            inv_links[link_title] = 'https://www.greateasternlife.com'+link['href']
    except:
        continue

In [31]:
inv_df = get_data(inv_links)
inv_df.head(10)

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.11it/s]


,page,text
0,Self-service guides/Investment-linked/Apportio...,Apportion premium for investment-linked plan s...
1,Self-service guides/Investment-linked/Apportio...,Apportion premium for investment-linked plan F...
2,Self-service guides/Investment-linked/Complete...,Complete Customer Knowledge Assessment (CKA) s...
3,Self-service guides/Investment-linked/Complete...,Complete Customer Knowledge Assessment (CKA) F...
4,Self-service guides/Investment-linked/Switch f...,Switch funds for an investment-linked plan ste...
5,Self-service guides/Investment-linked/Switch f...,Switch funds for an investment-linked plan FAQ...
6,Self-service guides/Investment-linked/Withdraw...,Withdraw from an investment-linked plan steps:...
7,Self-service guides/Investment-linked/Withdraw...,Withdraw from an investment-linked plan FAQ: H...


In [20]:
ins_r = requests.get('https://www.greateasternlife.com/sg/en/customer-services/guides.html?topic=insurance-guides')
ins_raw = BS(ins_r.content,'html.parser')
links_0 = ins_raw.find_all('a',class_='leo-button leo-font-normal leo-border-b-0 leo-button--link leo-link-arrow')
ins_links = {}
for link in links_0:
    try:
        if ('PDF' in link['title']
           ):
            link_title = 'Self-service guides/Insurance guides/'+link['title']
            ins_links[link_title] = 'https://www.greateasternlife.com'+link['href']
    except:
        continue

Temp - to create text files for OpenAI Assistant API:

In [32]:
# For the existing inv & claims dataframes
folder = r'ge_html_files_v1'
temp_list = [(inv_df,os.path.join(folder,'investment_linked.txt')),
           (claims_df,os.path.join(folder,'claims.txt'))]
for doc in temp_list:
    filename = doc[1]
    text = doc[0]['text'].tolist()
    with open(filename,'w') as file:
        for row in text:
            file.write(row+'\n')

In [75]:
# Loans, Payment, Policy assignment, Policy benefits, Policy reinstatement, Rewards and promotions
curated_r = requests.get('https://www.greateasternlife.com/sg/en/customer-services/guides.html?topic=rewards-and-promotions')
curated_raw = BS(curated_r.content,'html.parser')
curated_list = ['loans','payment','policy-assignment','policy-benefits','policy-reinstatement','policy-nomination','purchase-of-insurance','rewards-and-promotions']
divs = curated_raw.find_all('div')
divs = [div for div in divs if div.has_attr('class')]
divs = [div for div in divs if 'leo-p-lg' in div['class'] and 'list-item' in div['class'] and 'leo-text-accent' in div['class']]
curated_links = {}
for div in divs:
    a_list = div.find_all('a',class_="leo-button leo-font-normal leo-border-b-0 leo-button--link leo-link-arrow")
    for link in a_list:
        try:
            cat_1 = link['href'].split('/')[-2]
            cat_2 = link['href'].split('/')[-3]
            if ('PDF' not in link['title']
                and (cat_1 in curated_list
                or cat_2 in curated_list)
               ):
                if cat_1 in curated_list:
                    link_title = 'Self-service guides/'+cat_1+'/'+link['title']
                else: 
                    link_title = 'Self-service guides/'+cat_2+'/'+link['title']
                curated_links[link_title] = 'https://www.greateasternlife.com'+link['href']
        except:
            continue

In [77]:
folder = r'ge_html_files_v1'
for cat in tqdm(curated_list):
    temp_cat_list = [c for c in curated_links.keys() if cat in c]
    df = pd.DataFrame(columns = ['page','text'])
    for c in temp_cat_list:
        df0 = get_data(curated_links)   
        df = pd.concat([df,df0],ignore_index=True)
    text = df['text'].tolist()
    filename = os.path.join(folder,cat.replace('-','_')+'.txt')
    with open(filename,'w') as file:
        for row in text:
            file.write(row+'\n')
    df.to_json(cat.replace('-','_')+'.jsonl',lines=True,orient='records')

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [03:31<00:00, 26.39s/it]


In [91]:
inv_df.to_json('ge_ss_investmentlinked.jsonl',lines=True,orient='records')
claims_df.to_json('ge_ss_claims.jsonl',lines=True,orient='records')

Download links as pdf

In [1]:
import mechanicalsoup

In [23]:
folder = r'ge_html_files'
browser = mechanicalsoup.StatefulBrowser()
for links in tqdm([claims_links,inv_links,ins_links]):
    for file in links:
        if os.path.exists(folder) == False:
            os.makedirs(folder)
        pdf_path = os.path.join(folder,file.split('/')[-1]+'.pdf')
        response = browser.open(links[file])
        with open(pdf_path,'wb') as f:
            f.write(response.content)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:30<00:00, 50.32s/it]


### Retrieval

In [2]:
path2 = os.path.join(path,r'ge_html_files_jsonl')
data = pd.DataFrame(columns=['page','text'])
for f in os.listdir(path2):
#     if os.path.isfile(f):
    temp_df = pd.read_json(os.path.join(path2,f),lines=True,orient='records')
    data = pd.concat([data,temp_df],ignore_index=True)
texts = data['text'].tolist()

In [3]:
data.head()

,page,text
0,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim steps: If you...
1,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim steps: \nStep...
2,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim steps: \nStep...
3,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: Need mor...
4,Self-service guides/Claims/Careshield and Elde...,Careshield and Eldershield claim FAQ: What is ...


#### Train Retriever

In [4]:
# Init
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer, util, models

In [5]:
text_splitter_rc = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # To test
    chunk_overlap = 2,
    length_function = len,
    is_separator_regex = False
)
docs_rc = text_splitter_rc.create_documents(texts)
docs_rc_text = [doc.page_content for doc in docs_rc]

In [6]:
# Try models:

# emb_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2',device='cuda')
emb_model1 = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1',device='cuda')
emb_model1.save('SBERT_MiniLM_L6')
# emb_model1 = SentenceTransformer('SBERT_MiniLM_L6')
# emb_model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

C:\Users\yongz\miniforge3\envs\llm-env\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [20]:
docs_emb = emb_model1.encode(docs_rc_text) # Accepts iterable
# db = Chroma.from_documents(docs_rc,emb_model,persist_directory='./chroma_db_v0') # no 'embed_doc' method error
with open('ge_embeddings_v1.pkl','wb') as file:
    pickle.dump({'docs':docs_rc_text,'embeddings':docs_emb},file,protocol=pickle.HIGHEST_PROTOCOL)

After restarting a kernel, proceed to the following cells after init:

In [7]:
# emb_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2',device='cuda')
emb_model = SentenceTransformer('SBERT_MiniLM_L6',device='cuda')

In [8]:
with open('ge_embeddings_v1.pkl','rb') as file:
    stored_data = pickle.load(file)
    docs_rc_text = stored_data['docs']
    docs_emb = stored_data['embeddings']

In [9]:
test_queries = [
    'How do I make a Careshield Claim?',
    'How do I apply for a doctor\'s statement to get a disability claim?',
    'How can I file for personal accident claims for the Supreme plan?'
] 
def get_scores(query):
    query_emb = emb_model.encode(query)
#     scores = util.dot_score(query_emb,docs_emb)[0].cpu().tolist()
    scores = util.cos_sim(query_emb,docs_emb)[0].cpu().tolist()
    doc_score_pairs = list(zip(docs_rc_text,scores))
    doc_score_pairs = sorted(doc_score_pairs,key=lambda x: x[1], reverse=True)
    return doc_score_pairs # > 0.6

def get_docs(query,k=0.6):
    doc_scores = get_scores(query)
    docs = []
    for i in range(len(doc_scores)):
        if doc_scores[i][1] >= k:
            docs.append(doc_scores[i][0])
        else: break
    return docs

def get_prompt(query):
    docs = ''
    chosen_docs = get_docs(query)
    for i in chosen_docs:
        docs += i
        docs += '\n'
    prompt = f"You are a financial advisor from Great Eastern Singapore. You are to assist the client as professionally and truthfully as possible. Respond to the client\'s prompt based on the folowing documents:\n==== document(s): {docs}\n====\n\nprompt: {query}\nresponse:"
    return prompt

Chat Agent:

In [9]:
# Init
from openai import OpenAI
# from langchain_openai import ChatOpenAI
# from langchain.agents import tool
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# from dotenv import load_dotenv
# from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool #Error here

In [13]:
# load_dotenv
client = OpenAI(
    api_key = os.environ.get('OPENAI_API_KEY')
)
ge_prompt = get_prompt(test_queries[0])
response = client.chat.completions.create(
    messages = [
        {
        'role':'user',
        'content':ge_prompt,
        }  
    ],
    model="gpt-3.5-turbo"
)
print(response)

ChatCompletion(id='chatcmpl-8orS54ZLFu9mKfKpHakDr9c2sUTN7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To make a Careshield claim, you need to follow these steps:\n\nStep 1: Download the necessary documents from our website. These documents include the Claimant's Statement, Letter of Undertaking and Indemnity, and Clinical Abstract Application.\n\nStep 2: Book an appointment with one of our appointed Assessors. You will need to be assessed by the Assessor to obtain the Assessor's Statement. You can choose between a clinic appointment or a house call, but there will be a fee of $100 for a clinic-based assessment or $250 for a non-clinic based assessment.\n\nStep 3: On the appointment day, bring along the completed documents you downloaded in Step 1, as well as any available medical reports or discharge summary.\n\nStep 4: The Assessor's Statement and all your documents will be sent directly to Great Eastern. Once we receive the d

In [19]:
print('Query:',test_queries[0],'\n')
# print(get_scores(test_queries[0])[0:2])
print('Response:',response.choices[0].message.content)

Query: How do I make a Careshield Claim? 

Response: To make a Careshield claim, you need to follow these steps:

Step 1: Download the necessary documents from our website. These documents include the Claimant's Statement, Letter of Undertaking and Indemnity, and Clinical Abstract Application.

Step 2: Book an appointment with one of our appointed Assessors. You will need to be assessed by the Assessor to obtain the Assessor's Statement. You can choose between a clinic appointment or a house call, but there will be a fee of $100 for a clinic-based assessment or $250 for a non-clinic based assessment.

Step 3: On the appointment day, bring along the completed documents you downloaded in Step 1, as well as any available medical reports or discharge summary.

Step 4: The Assessor's Statement and all your documents will be sent directly to Great Eastern. Once we receive the documents, we will proceed to process your claim.

It's important to note that if you are unable to go back to Singap